In [1]:
import pyreadr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import pandas as pd
import math
import reverse_geocoder as rg
from matplotlib.patches import Polygon
import bokeh
import json
from bokeh.io import show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider, Column,BooleanFilter)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure
from bokeh.models import Range1d
from shapely.geometry import Point

import bokeh as bokeh
from bokeh.models import Slider
from ipywidgets import interact, FloatSlider
from bokeh.io import output_notebook, show, push_notebook
from bokeh.plotting import figure
output_notebook(hide_banner=True)

In [2]:
result = pyreadr.read_r('../Data/data_train_DF.RData') 
df1 = result["data_train_DF"]
df1['BA'] = df1['BA'].fillna(0)
df1['CNT'] = df1['CNT'].fillna(0)

#read shp file containing polygons of the US states 
#source: https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html
states1=gpd.read_file("../Data/cb_2018_us_state_5m/cb_2018_us_state_5m.shp")

#extract
wf=df1[df1['year']==1993]
wf=wf[wf['month']==3]

In [3]:
l=[]
for i in range(len(wf)):
    p=(wf.lat.iloc[i],wf.lon.iloc[i])
    l.append(p)
coor=l
res=rg.search(coor)#search where the location belongs to
for i in res:
    if i["admin1"]=='Tamaulipas':
        i["admin1"]='Texas'
    if i["admin1"]=='British Columbia':
        i["admin1"]='Washington'
    if i["admin1"]=='Ontario':
        i["admin1"]="Minnesota"
    if i["admin1"]=='Chihuahua':
        i["admin1"]="New Mexico"
    if i["admin1"]=='Coahuila':
        i["admin1"]="Texas"
    if i["admin1"]=='New Brunswick':
        i["admin1"]="Maine"
    if i["admin1"]=='Sonora':
        i["admin1"]='Arizona'
    if i["admin1"]=='Baja California':
        i["admin1"]='California'
    if i["admin1"]=='Quebec':
        i["admin1"]="New York"

Loading formatted geocoded file...


In [4]:
#useful dictionaries

#source from https://www.census.gov/geographies/reference-files/2010/geo/state-area.html
totarea={
    "Alabama": 135767,
    "Arizona": 295234,
    "Arkansas": 137732,
    "California": 423967,
    "Colorado": 269601,
    "Connecticut": 14357,
    "Delaware": 6446,
    "Florida": 170312,
    "Georgia": 153910,
    "Idaho": 216443,
    "Illinois": 149995,
    "Indiana": 94326,
    "Iowa": 145746,
    "Kansas": 213100,
    "Kentucky": 104656,
    "Louisiana": 135659,
    "Maine": 91633,
    "Maryland": 32131,
    "Massachusetts": 27336,
    "Michigan": 250487,
    "Minnesota": 225163,
    "Mississippi": 125438,
    "Missouri": 180540,
    "Montana": 380831,
    "Nebraska": 200330,
    "Nevada": 286380,
    "New Hampshire": 24214,
    "New Jersey": 22591,
    "New Mexico": 314917,
    "New York": 141297,
    "North Carolina": 139391,
    "North Dakota": 183108,
    "Ohio": 116098,
    "Oklahoma": 181037,
    "Oregon": 254799,
    "Pennsylvania": 119280,
    "Rhode Island": 4001,
    "South Carolina": 82933,
    "South Dakota": 199729,
    "Tennessee": 109153,
    "Texas": 695662,
    "Utah": 219882,
    "Vermont": 24906,
    "Virginia": 110787,
    "Washington": 184661,
    "West Virginia": 62756,
    "Wisconsin": 169635,
    "Wyoming": 253335}

In [10]:
def count_init():
    d={
    "Alabama": 0,
    "Alaska": 0,
    "Arizona": 0,
    "Arkansas": 0,
    "California": 0,
    "Colorado": 0,
    "Connecticut": 0,
    "District of Columbia": 0,
    "Delaware": 0,
    "Florida": 0,
    "Georgia": 0,
    "Hawaii": 0,
    "Idaho": 0,
    "Illinois": 0,
    "Indiana": 0,
    "Iowa": 0,
    "Kansas": 0,
    "Kentucky": 0,
    "Louisiana": 0,
    "Maine": 0,
    "Maryland": 0,
    "Massachusetts": 0,
    "Michigan": 0,
    "Minnesota": 0,
    "Mississippi": 0,
    "Missouri": 0,
    "Montana": 0,
    "Nebraska": 0,
    "Nevada": 0,
    "New Hampshire": 0,
    "New Jersey": 0,
    "New Mexico": 0,
    "New York": 0,
    "North Carolina": 0,
    "North Dakota": 0,
    "Ohio": 0,
    "Oklahoma": 0,
    "Oregon": 0,
    "Pennsylvania": 0,
    "Rhode Island": 0,
    "South Carolina": 0,
    "South Dakota": 0,
    "Tennessee": 0,
    "Texas": 0,
    "Utah": 0,
    "Vermont": 0,
    "Virginia": 0,
    "Washington": 0,
    "West Virginia": 0,
    "Wisconsin": 0,
    "Wyoming": 0}
    return d


def countfire(res,df):
    stcount=count_init()
    for i in range(len(res)):
        if res[i]['admin1'] in totarea.keys():
            stcount[res[i]['admin1']]+=df.CNT.iloc[i]/totarea[res[i]['admin1']]*100000
    return stcount

def areafire(res,df):
    stcount=count_init()
    for i in range(len(res)):
        if res[i]['admin1'] in totarea.keys():
            stcount[res[i]['admin1']]+=df.BA.iloc[i]/totarea[res[i]['admin1']]*100000
    return stcount

In [11]:
class states: #to avoid 'DataFrame' object has no attribute 'dtype' errors, and overwriting
    def __init__(self):
        self.states=gpd.read_file("../Data/cb_2018_us_state_5m/cb_2018_us_state_5m.shp")#have to recall for each update..?
        
    def add_column(self,colnum,colname,col):
        self.states.insert(colnum,colname,col,True)
        
    def delete_column(self,colname):
        self.states=self.states.drop(columns=[colname])

In [12]:
#inspired by: 
#- https://towardsdatascience.com/walkthrough-mapping-basics-with-bokeh-and-geopandas-in-python-43f40aa5b7e9
#- https://stackoverflow.com/questions/55362916/how-to-pass-the-slider-value-in-bokeh-back-to-python-code

s=states()

#slider1 = Slider(title = "Month", start = 3, end = 9, step = 1, value = 3)
#slider2= Slider(title="year",start=1993, end=2015, step=1,value=1993)

#month=slider1.value
#year=slider2.value


def update(year,month,opacity,CNTBA):

#view = CDSView(source = sitesource, filters = [indices])
    extract=df1[ (df1["year"]==year) & (df1["month"]==month) ]
    lba=[]
    ba=areafire(res,extract)
    dba=dict({'States':list(ba.keys()),'Number':list(ba.values())})
    dba2=pd.DataFrame.from_dict(dba)
    for i in range(len(s.states)):
        if s.states["NAME"].iloc[i] in dba["States"]:
            lba.append(math.log(ba[s.states["NAME"].iloc[i]]+1))
        else:
            lba.append(0)
    s.add_column(2,"ba",lba)
    geosource = GeoJSONDataSource(geojson = s.states.to_json())
    s.delete_column("ba")

# Define color palettes
    palette = brewer['YlGnBu'][8]
    palette = palette[::-1] # reverse order of colors so higher values have darker colors
# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
    color_mapper = LinearColorMapper(palette = palette, low = 0, high = 8)
# Define custom tick labels for color bar.
    tick_labels = {"0": "2⁰", "1": "2¹",
     "2":"2²", "3":"2³",
     "4":"2⁴", "5":"2⁵",
     "6":"2⁶", "7":"2⁷",
     "8":"2⁸+"}
# Create color bar.
    color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 8,
                     width = 500, height = 20,
                     border_line_color = None,
                     location = (0,0), 
                     orientation = "horizontal",
                     major_label_overrides = tick_labels)

#add legend for size

# Create figure object.
    p = figure(title = 'Wildfire in 0'+str(int(month))+'/'+str(int(year)), 
           plot_height = 600 ,
           plot_width = 950, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.x_range=Range1d(-127, -65)
    p.y_range=Range1d(23, 50)
# Add patch renderer to figure.
    states1 = p.patches('xs','ys', source = geosource,
                   fill_color = {"field" :'ba',
                                 "transform" : color_mapper},
                   line_color = "gray", 
                   line_width = 0.25, 
                   fill_alpha = 1)
    
    #indices=[]
    #for i in range(len(df1)):
    #    if (df1["month"].iloc[i]==month) & (df1["year"].iloc[i]==year):
    #        indices.append(True)
    #    else:
    #        indices.append(False)
            
    #view=CDSView(source = sitesource, filters = [BooleanFilter(indices)])
    #sites=p.circle('x', 'y', source=sitesource, color = 'red', size = 'CNT', alpha = opacity,view=view)
    if CNTBA==0:
        extract=extract[extract["CNT"]!=0]
        extract["logCNT"]=[math.log(i+1)*2 for i in extract['CNT']]
        sites=p.circle(extract['lon'],extract['lat'],color = 'red', size = extract['logCNT'],alpha = opacity)
    else:
        extract=extract[extract["BA"]!=0]
        extract["logBA"]=[math.log(i+1)*2 for i in extract['BA']]
        sites=p.circle(extract['lon'],extract['lat'],color = 'red', size = extract['logBA'],alpha = opacity)
# Create hover tool
    p.add_tools(HoverTool(renderers = [states1],
                      tooltips = [('State','@NAME'),
                                ('Burnt Area','@ba per 100000km²')]))
    #if opacity>0:
    #    p.add_tools(HoverTool(renderers = [sites],
    #                      tooltips = [('Burnt Area', '@BA'),
    #                                  ('Number of Wildfires', '@CNT'),
    #                                  ('Latitude', '@lat'),
    #                                 ('Longitude','@lon')]))

    p.add_layout(color_bar, "below")
    output_notebook()
    show(p,notebook_handle=True)
    
    push_notebook()

_ = interact(update, year =FloatSlider(min=1993, max=2015,
                                     step=1, value= 1993),
             month=FloatSlider(min=3, max=9, step=1, value= 3),
             opacity=FloatSlider(min=0, max=0.5, step=.1, value= .5),
            CNTBA=FloatSlider(min=0,max=1,step=1,value=0))

interactive(children=(FloatSlider(value=1993.0, description='year', max=2015.0, min=1993.0, step=1.0), FloatSl…